<a href="https://colab.research.google.com/github/pbhuch/GL_DecA_G4_NLP1/blob/NLP1_PM/Capstone_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Project Modelling part**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ftfy

     |████████████████████████████████| 64 kB 2.1 MB/s 
  Created wheel for ftfy: filename=ftfy-6.0.3-py3-none-any.whl size=41933 sha256=47efd8168479d574774f37aca6743445039dce0ce0556858f15c2bb45d9e4678
  Stored in directory: /root/.cache/pip/wheels/19/f5/38/273eb3b5e76dfd850619312f693716ac4518b498f5ffb6f56d
Successfully built ftfy


In [ ]:
# importing libraries
from time import time
from PIL import Image
from zipfile import ZipFile
import os, sys, itertools, re
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
import plotly as py
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot, plot
from sklearn.preprocessing import QuantileTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score, confusion_matrix, classification_report
import sklearn.neighbors._base
#import sys
#sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
import imblearn
from imblearn.over_sampling import SMOTE

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Activation, Conv2D, MaxPooling2D, Reshape, Embedding, LSTM,  TimeDistributed, Bidirectional, Lambda, Input, Add, GlobalMaxPool1D
from tensorflow.keras import regularizers, optimizers
from sklearn.metrics import r2_score
from tensorflow.keras.models import load_model
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
#import cv2
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# to define loss
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.backend import log, epsilon

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.datasets import imdb

from itertools import islice

import re
import nltk
from nltk.corpus import stopwords

from ftfy import fix_encoding, fix_text, badness
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

from sklearn.utils import resample

import pickle, string

import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot

import spacy

# Suppress warnings
import warnings; warnings.filterwarnings('ignore')

SEED = 123                 # to be able to rerun the same NN
np.random.seed(SEED)
tf.random.set_seed(SEED)

from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [ ]:
import os; 
os.chdir('/content/drive/MyDrive/Great_Learning/Capstone')

In [ ]:
dataset = pd.read_csv("dataset1_mojibaked_LangDetect_NewColumnAdded.csv", encoding = 'utf-8')

In [ ]:
dataset.head()

,Unnamed: 0,Short description,Description,Caller,Assignment group,combined_description_caller,combined_description,ConvertedToEnglish,Language
0,0,login issue,verified user detailsemployee manager name che...,spxjnwir pjlcoqds,GRP_0,login issue verified user detailsemployee mana...,login issue verified user detailsemployee mana...,NaN,en
1,1,outlook,received from hmjdrvpb.komuaywn@gmail.com\n he...,hmjdrvpb komuaywn,GRP_0,outlook received from hmjdrvpb.komuaywn@gmail....,outlook received from hmjdrvpb.komuaywn@gmail....,NaN,en
2,2,cant log in to vpn,received from eylqgodm.ybqkwiam@gmail.com\n hi...,eylqgodm ybqkwiam,GRP_0,cant log in to vpn received from eylqgodm.ybqk...,cant log in to vpn received from eylqgodm.ybqk...,NaN,en
3,3,unable to access hrtool page,unable to access hrtool page,xbkucsvz gcpydteq,GRP_0,unable to access hrtool page unable to access ...,unable to access hrtool page unable to access ...,NaN,en
4,4,skype error,skype error,owlgqjme qhcozdfx,GRP_0,skype error skype error owlgqjme qhcozdfx,skype error skype error,NaN,no


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8417 entries, 0 to 8416
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   8417 non-null   int64  
 1   Short description            8408 non-null   object 
 2   Description                  8413 non-null   object 
 3   Caller                       8417 non-null   object 
 4   Assignment group             8417 non-null   object 
 5   combined_description_caller  8417 non-null   object 
 6   combined_description         8417 non-null   object 
 7   ConvertedToEnglish           0 non-null      float64
 8   Language                     8417 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 591.9+ KB


In [ ]:
dataset['Assignment group'].value_counts()


GRP_0     3934
GRP_8      645
GRP_24     285
GRP_12     257
GRP_9      252
GRP_2      241
GRP_19     215
GRP_3      200
GRP_6      183
GRP_13     145
GRP_10     140
GRP_5      128
GRP_14     118
GRP_25     116
GRP_33     107
GRP_4      100
GRP_29      97
GRP_18      88
GRP_16      85
GRP_31      69
GRP_17      68
GRP_7       68
GRP_34      62
GRP_26      56
GRP_40      45
GRP_28      44
GRP_41      40
GRP_30      39
GRP_15      38
GRP_42      37
GRP_20      36
GRP_45      35
GRP_22      31
GRP_1       31
GRP_11      30
GRP_21      28
GRP_47      27
GRP_23      25
GRP_48      25
GRP_62      25
GRP_39      19
GRP_27      18
GRP_37      16
GRP_60      16
GRP_36      15
GRP_44      15
GRP_50      14
GRP_65      11
GRP_53      11
GRP_52       9
GRP_51       8
GRP_55       8
GRP_59       6
GRP_49       6
GRP_46       6
GRP_43       5
GRP_32       4
GRP_66       4
GRP_63       3
GRP_58       3
GRP_68       3
GRP_38       3
GRP_56       3
GRP_54       2
GRP_69       2
GRP_71       2
GRP_57    

In [ ]:
dataset['Language'].value_counts()

en       7010
de        415
af        271
it        121
fr        105
no         71
nl         68
sv         53
zh-cn      49
es         47
ca         39
pl         27
da         25
ko         17
pt         17
cy         11
tl         11
ro         10
et         10
sq          8
hr          6
sl          6
fi          5
vi          3
id          3
so          2
cs          2
lt          2
ja          1
tr          1
hu          1
Name: Language, dtype: int64

In [ ]:
data_eng = dataset[dataset['Language']=='en']

In [ ]:
data_eng.head()

,Unnamed: 0,Short description,Description,Caller,Assignment group,combined_description_caller,combined_description,ConvertedToEnglish,Language
0,0,login issue,verified user detailsemployee manager name che...,spxjnwir pjlcoqds,GRP_0,login issue verified user detailsemployee mana...,login issue verified user detailsemployee mana...,NaN,en
1,1,outlook,received from hmjdrvpb.komuaywn@gmail.com\n he...,hmjdrvpb komuaywn,GRP_0,outlook received from hmjdrvpb.komuaywn@gmail....,outlook received from hmjdrvpb.komuaywn@gmail....,NaN,en
2,2,cant log in to vpn,received from eylqgodm.ybqkwiam@gmail.com\n hi...,eylqgodm ybqkwiam,GRP_0,cant log in to vpn received from eylqgodm.ybqk...,cant log in to vpn received from eylqgodm.ybqk...,NaN,en
3,3,unable to access hrtool page,unable to access hrtool page,xbkucsvz gcpydteq,GRP_0,unable to access hrtool page unable to access ...,unable to access hrtool page unable to access ...,NaN,en
5,5,unable to log in to engineering tool and skype,unable to log in to engineering tool and skype,eflahbxn ltdgrvkz,GRP_0,unable to log in to engineering tool and skype...,unable to log in to engineering tool and skype...,NaN,en


In [ ]:
# Create a target categorical column
data_eng['target'] = data_eng['Assignment group'].astype('category').cat.codes
data_eng.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7010 entries, 0 to 8415
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   7010 non-null   int64  
 1   Short description            7002 non-null   object 
 2   Description                  7006 non-null   object 
 3   Caller                       7010 non-null   object 
 4   Assignment group             7010 non-null   object 
 5   combined_description_caller  7010 non-null   object 
 6   combined_description         7010 non-null   object 
 7   ConvertedToEnglish           0 non-null      float64
 8   Language                     7010 non-null   object 
 9   target                       7010 non-null   int8   
dtypes: float64(1), int64(1), int8(1), object(7)
memory usage: 554.5+ KB


In [ ]:
data_eng['target'].value_counts()

0     3138
72     642
73     249
12     235
4      224
11     199
23     194
56     180
5      142
2      134
45     126
6      115
34     100
18      97
22      94
10      87
8       83
67      68
9       65
17      59
19      53
28      47
35      42
36      40
7       36
40      34
13      34
21      34
25      33
15      31
3       29
1       29
14      28
27      28
42      27
16      24
33      19
57      16
31      16
20      16
39      14
46      13
30      13
62      11
49      11
59      10
48       8
51       8
24       8
47       8
38       5
55       5
37       4
63       4
41       4
60       3
44       3
65       3
32       3
54       3
52       3
69       2
53       2
26       2
50       2
66       2
43       2
64       1
68       1
29       1
61       1
58       1
70       1
71       1
Name: target, dtype: int64

In [ ]:
# A class that logs the time
class Timer():
    '''
    A generic class to log the time
    '''
    def __init__(self):
        self.start_time = None
    def start(self):
        self.start_time = time()
    def stop(self):
        return 'Time taken: %2fs' % (time()-self.start_time)
    
timer = Timer()

# A method that plots the Precision-Recall curve
def plot_prec_recall_vs_thresh(precisions, recalls, thresholds):
    plt.figure(figsize=(15,10))
    plt.plot(thresholds, precisions[:-1], 'b--', label='precision')
    plt.plot(thresholds, recalls[:-1], 'g--', label = 'recall')
    plt.xlabel('Threshold')
    plt.legend()

    



# A method to train and test the model
def run_ML(estimator, X_train, X_test, y_train, y_test, arch_name=None, pipelineRequired=True, isDeepModel=False):
    timer.start()
    # train the model
    clf = estimator

    if pipelineRequired :
        clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', estimator),
                     ])
      
    if isDeepModel:
        clf.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=10, batch_size=128,verbose=1,callbacks=call_backs(arch_name))
        # predict from the claffier
        y_pred = clf.predict(X_test)
        y_pred = np.argmax(y_pred, axis=1)
        y_train_pred = clf.predict(X_train) # predict target for train
        y_train_pred = np.argmax(y_train_pred, axis=1)
    else :
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test) # predict target for test
        y_train_pred = clf.predict(X_train)
    
    print('Estimator:', clf)
    print('='*80)
    print('Training accuracy: %.2f%%' % (accuracy_score(y_train,y_train_pred) * 100))
    print('Testing accuracy: %.2f%%' % (accuracy_score(y_test, y_pred) * 100))
    print('='*80)
    print('Confusion matrix:\n %s' % (confusion_matrix(y_test, y_pred)))
    print('='*80)
    print('Classification report:\n %s' % (classification_report(y_test, y_pred)))
    print(timer.stop(), 'to run the model')



In [ ]:
# Create training and test datasets with 80:20 ratio
X_train, X_test, y_train, y_test = train_test_split(data_eng.combined_description, 
                                                    data_eng.target, 
                                                    test_size=0.20, 
                                                    random_state=42)
print('\033[1mShape of the training set:\033[0m', X_train.shape, X_test.shape)
print('\033[1mShape of the test set:\033[0m', y_train.shape, y_test.shape)

Shape of the training set: (5608,) (1402,)
Shape of the test set: (5608,) (1402,)


In [ ]:
# ML Modeling
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

**Naive Bayes Classifier**

In [ ]:
run_ML(MultinomialNB(), X_train, X_test, y_train, y_test)


Estimator: Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])
Training accuracy: 54.21%
Testing accuracy: 53.92%
Confusion matrix:
 [[629   0   0 ...   0   0   0]
 [  3   0   0 ...   0   2   0]
 [ 21   0   0 ...   0  15   0]
 ...
 [  1   0   0 ...   0   0   0]
 [ 10   0   0 ...   0 123   0]
 [ 16   0   0 ...   0  32   0]]
Classification report:
               precision    recall  f1-score   support

           0       0.55      1.00      0.71       629
           1       0.00      0.00      0.00         8
           2       0.00      0.00      0.00        36
           3       0.00      0.00      0.00         6
           4       0.33      0.10      0.15        41
           5       0.00      0.00      0.00        21
           6       0.00      0.00      0.00        28
           7       0.00      0.00      0.00        11
           8       0.00      0.00      0.00        13
           9       0.00      0.00      0.00

In [ ]:
run_ML(KNeighborsClassifier(), X_train, X_test, y_train, y_test)

Estimator: Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', KNeighborsClassifier())])
Training accuracy: 68.38%
Testing accuracy: 63.12%
Confusion matrix:
 [[616   0   0 ...   0   1   1]
 [  1   0   0 ...   0   0   0]
 [ 13   0  15 ...   0   4   0]
 ...
 [  1   0   0 ...   0   0   0]
 [  5   0   1 ...   0 106   3]
 [ 11   0   0 ...   0  25   7]]
Classification report:
               precision    recall  f1-score   support

           0       0.64      0.98      0.77       629
           1       0.00      0.00      0.00         8
           2       0.83      0.42      0.56        36
           3       1.00      0.33      0.50         6
           4       0.42      0.32      0.36        41
           5       0.43      0.14      0.21        21
           6       0.60      0.43      0.50        28
           7       0.50      0.09      0.15        11
           8       0.00      0.00      0.00        13
           9       1.00      0.81   

In [ ]:
# SVM with Linear kernel
run_ML(LinearSVC(), X_train, X_test, y_train, y_test)

Estimator: Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LinearSVC())])
Training accuracy: 92.96%
Testing accuracy: 67.97%
Confusion matrix:
 [[579   0   1 ...   0   0   3]
 [  0   0   0 ...   0   1   0]
 [  5   0  22 ...   0   4   1]
 ...
 [  1   0   0 ...   0   0   0]
 [  4   0   1 ...   0 115   0]
 [  4   0   0 ...   0  30  13]]
Classification report:
               precision    recall  f1-score   support

           0       0.74      0.92      0.82       629
           1       0.00      0.00      0.00         8
           2       0.81      0.61      0.70        36
           3       1.00      0.33      0.50         6
           4       0.62      0.61      0.62        41
           5       0.40      0.48      0.43        21
           6       0.61      0.50      0.55        28
           7       0.67      0.18      0.29        11
           8       0.25      0.08      0.12        13
           9       1.00      1.00      1.00    

In [ ]:
run_ML(RandomForestClassifier(n_estimators=100), X_train, X_test, y_train, y_test)

Estimator: Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier())])
Training accuracy: 94.72%
Testing accuracy: 61.91%
Confusion matrix:
 [[629   0   0 ...   0   0   0]
 [  0   0   0 ...   0   3   0]
 [ 19   0  11 ...   0   4   0]
 ...
 [  1   0   0 ...   0   0   0]
 [  6   0   1 ...   0 117   4]
 [ 16   0   0 ...   0  25   7]]
Classification report:
               precision    recall  f1-score   support

           0       0.60      1.00      0.75       629
           1       0.00      0.00      0.00         8
           2       0.92      0.31      0.46        36
           3       0.00      0.00      0.00         6
           4       0.63      0.29      0.40        41
           5       0.20      0.05      0.08        21
           6       0.77      0.36      0.49        28
           7       0.00      0.00      0.00        11
           8       0.00      0.00      0.00        13
           9       1.00      0.88 

In [ ]:
run_ML(XGBClassifier(n_estimators=100), X_train, X_test, y_train, y_test)

Estimator: Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', XGBClassifier(objective='multi:softprob'))])
Training accuracy: 80.28%
Testing accuracy: 62.20%
Confusion matrix:
 [[586   0   0 ...   0   2   1]
 [  0   0   0 ...   0   2   0]
 [ 13   0  12 ...   0   4   0]
 ...
 [  1   0   0 ...   0   0   0]
 [  6   0   1 ...   0 110   0]
 [ 11   0   0 ...   0  30   6]]
Classification report:
               precision    recall  f1-score   support

           0       0.65      0.93      0.77       629
           1       0.00      0.00      0.00         8
           2       0.86      0.33      0.48        36
           3       0.25      0.33      0.29         6
           4       0.50      0.37      0.42        41
           5       0.50      0.33      0.40        21
           6       0.67      0.29      0.40        28
           7       0.75      0.27      0.40        11
           8       0.45      0.38      0.42        13
           9     